In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Use of the `HSDetection` class

In [ ]:
import sys
sys.path.append('..')
from hs2 import HSDetection
from probe import HierlmannVisapyEmulationProbe
import numpy as np
import h5py

In [ ]:
# raw data location
data_path = 'rawHierlmannVisapy.npy'

# detection parameters
to_localize = True
cutout_start = 10
cutout_end = 20
threshold = 12
file_name = 'ProcessedSpikes_visapy'

In [ ]:
Probe = HierlmannVisapyEmulationProbe(data_path)
H = HSDetection(Probe, to_localize, cutout_start, cutout_end, threshold,
                maa=0, maxsl=12, minsl=3, ahpthr=0, out_file_name=file_name, save_all=True)

In [ ]:
Probe.show()

In [ ]:
H.DetectFromRaw()

In [ ]:
H.LoadDetected()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
H.PlotTracesChannels(510)

In [ ]:
def closestChannels(neurons):
    closest_channels = []
    for neuron in neurons:
        dists = np.sqrt((elec_x-somapos[0][neuron][0])**2+(elec_y-somapos[0][neuron][1])**2)
        chinds = np.argsort(dists) # sort by distance, first is closest
        closest_channels.append((neuron, chinds[0]))
    return closest_channels
plt.figure(figsize=(15, 10))
print(closestChannels([50])[0][1])
H.PlotData(50, 10308, closestChannels([13])[0][1])

# Use of the `HSClustering` class

In [ ]:
from hs2 import HSClustering
from sklearn.cluster import DBSCAN

In [ ]:
# Load from file
#C = Clustering(['results/ProcessedSpikes_mea1k.bin', 'results/ProcessedSpikes_mea1k.bin'], cutout_length=41)

# Or if the spikes are already in memory
# simply load from the Detection class
C = HSClustering(H)

In [ ]:
C.ShapePCA(pca_ncomponents=2, pca_whiten=True)

C.CombinedClustering(alpha=0.4, bandwidth=0.3, bin_seeding=True,
                     min_bin_freq=10, n_jobs=-1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
C.PlotAll(invert=True, s=1, show_labels=True)
plt.title("MeanShift, bandwidth=.3, min_bin_freq=10");
plt.xlim([0,25])
plt.ylim([0,25])

In [ ]:
C.SaveHDF5("f1.hdf5")